In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  
%load_ext autoreload
%autoreload 2
from librep.estimators.simclr.torch.standartized_balanced import StandardizedBalancedDataset
import os,torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def get_data(dataset_name,sensors,normalize_data):    
    working_directory=f"result/{dataset_name}/"
    data_folder = f"../../../../../../experiment_executor/data/standartized_balanced/{dataset_name}/"
    dataset = StandardizedBalancedDataset(data_folder, sensors=sensors)
    X_train, y_train,X_test, y_test,X_val, y_val = dataset.get_all_data(normalize_data=normalize_data, resize_data=False)
    return X_train, y_train,X_test, y_test,X_val, y_val
        #print(f"shape: X_train {X_train.shape} --- X_test {X_test.shape} --- X_test {X_val.shape}")

In [2]:
X_train, y_train,X_test, y_test,X_val, y_val=get_data("MotionSense",['accel','gyro'],False)
input_shape=X_train[0].shape
print(input_shape)

(360,)


In [3]:
batch_size_head = 256
decay_steps_head = 1000
temperature_head = 0.77
n_components=60
epochs_head = 300
input_shape=[60,6]
transform_funcs=['noise_vectorized', 'channel_shuffle_vectorized', 'amplify_attenuate']
verbose=1
import os,torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [4]:
import simclr_utils as sm_ut
transformations=sm_ut.transform_combinations(3,4)

print(transformations)

[['noise_vectorized', 'scaling_vectorized', 'rotation_vectorized'], ['noise_vectorized', 'scaling_vectorized', 'negate_vectorized'], ['noise_vectorized', 'scaling_vectorized', 'time_flip_vectorized'], ['noise_vectorized', 'scaling_vectorized', 'channel_shuffle_vectorized'], ['noise_vectorized', 'scaling_vectorized', 'time_segment_permutation'], ['noise_vectorized', 'scaling_vectorized', 'time_shift'], ['noise_vectorized', 'scaling_vectorized', 'amplify_attenuate'], ['noise_vectorized', 'scaling_vectorized', 'add_random_noise'], ['noise_vectorized', 'scaling_vectorized', 'random_phase_shift'], ['noise_vectorized', 'scaling_vectorized', 'spectral_distortion'], ['noise_vectorized', 'scaling_vectorized', 'phase_modulation'], ['noise_vectorized', 'rotation_vectorized', 'negate_vectorized'], ['noise_vectorized', 'rotation_vectorized', 'time_flip_vectorized'], ['noise_vectorized', 'rotation_vectorized', 'channel_shuffle_vectorized'], ['noise_vectorized', 'rotation_vectorized', 'time_segment_p

In [5]:
from librep.estimators.simclr.torch.simclr_linear_estimator import Simclr_Linear_Estimator
#from librep.estimators.simclr.torch.simclr_full_estimator import Simclr_Linear_Estimator


In [6]:
linear_model=Simclr_Linear_Estimator(input_shape=input_shape,
                        n_components=n_components,
                        batch_size_head=batch_size_head,
                        transform_funcs=transform_funcs,
                        temperature_head=temperature_head, 
                        epochs_head=epochs_head,
                        save_model=False,
                        verbose=1,
                        total_epochs=50,
                        batch_size=32,
                        lr=0.001)

In [7]:
linear_model.fit(X_train,y_train,None,None)



(2846, 60, 6) [60, 6]
epoch: 1 loss: 5.551
epoch: 2 loss: 5.504
epoch: 3 loss: 5.488
epoch: 4 loss: 5.463
epoch: 5 loss: 5.426
epoch: 6 loss: 5.428
epoch: 7 loss: 5.301
epoch: 8 loss: 5.211
epoch: 9 loss: 5.185
epoch: 10 loss: 5.150
epoch: 11 loss: 5.163
epoch: 12 loss: 5.147
epoch: 13 loss: 5.124
epoch: 14 loss: 5.130
epoch: 15 loss: 5.112
epoch: 16 loss: 5.106
epoch: 17 loss: 5.090
epoch: 18 loss: 5.099
epoch: 19 loss: 5.084
epoch: 20 loss: 5.078
epoch: 21 loss: 5.087
epoch: 22 loss: 5.068
epoch: 23 loss: 5.055
epoch: 24 loss: 5.062
epoch: 25 loss: 5.050
epoch: 26 loss: 5.053
epoch: 27 loss: 5.042
epoch: 28 loss: 5.059
epoch: 29 loss: 5.044
epoch: 30 loss: 5.035
epoch: 31 loss: 5.063
epoch: 32 loss: 5.046
epoch: 33 loss: 5.056
epoch: 34 loss: 5.051
epoch: 35 loss: 5.063
epoch: 36 loss: 5.039
epoch: 37 loss: 5.054
epoch: 38 loss: 5.057
epoch: 39 loss: 5.041
epoch: 40 loss: 5.038
Early stopping after 40 epochs with no improvement.
Epoch [1/50] - Loss: 1.0510261537653676
Val Accuracy: 0

In [8]:
print(linear_model.model)

LinearModel_Intermediate(
  (base_model): BaseModel(
    (conv1): Conv1d(6, 32, kernel_size=(24,), stride=(1,))
    (dropout1): Dropout(p=0.1, inplace=False)
    (conv2): Conv1d(32, 64, kernel_size=(16,), stride=(1,))
    (dropout2): Dropout(p=0.1, inplace=False)
    (conv3): Conv1d(64, 60, kernel_size=(8,), stride=(1,))
    (dropout3): Dropout(p=0.1, inplace=False)
    (global_max_pooling): AdaptiveMaxPool1d(output_size=1)
    (relu): ReLU()
  )
  (linear): Linear(in_features=60, out_features=6, bias=True)
)
